# Symptom Disease
*Lab 17 May 2022*

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS SD;

CREATE TABLE SD(
    source VARCHAR,
    target VARCHAR,
    weight FLOAT
) AS SELECT source,target,weight
FROM CSVREAD('ncomms5212-s5_network.csv');

In [3]:
SELECT * FROM SD LIMIT 5;

In [4]:
CALL CSVWRITE('symptoms-disease-g08-edges.csv', 'SELECT * FROM SD WHERE weight>=0.7', 'fieldDelimiter=');

3248

In [5]:
DROP TABLE IF EXISTS Mesh;

CREATE TABLE Mesh(
    term VARCHAR,
    tree VARCHAR,
    l1 VARCHAR(3),
    l2 VARCHAR(3),
    l3 VARCHAR(3),
    l4 VARCHAR(3),
    l5 VARCHAR(3),
    l6 VARCHAR(3),
    l7 VARCHAR(3),
    PRIMARY KEY (tree)
) AS SELECT term,tree,l1,l2,l3,l4,l5,l6,l7
FROM CSVREAD('mesh-code-tree.csv');

In [6]:
DROP TABLE IF EXISTS MeshFirst;

CREATE TABLE MeshFirst(
    term VARCHAR,
    tree VARCHAR,
    PRIMARY KEY(term)
) AS SELECT term,MIN(tree) tree
     FROM Mesh
     GROUP BY term;

In [7]:
CREATE TABLE MeshDedup(
    term VARCHAR,
    tree VARCHAR,
    l1 VARCHAR(3),
    l2 VARCHAR(3),
    l3 VARCHAR(3),
    l4 VARCHAR(3),
    l5 VARCHAR(3),
    l6 VARCHAR(3),
    l7 VARCHAR(3),
    PRIMARY KEY (term)
) AS SELECT MF.term,MF.tree,M.l1,M.l2,M.l3,M.l4,M.l5,M.l6,M.l7
  FROM MeshFirst MF, Mesh M
  WHERE MF.tree=M.tree;

In [8]:
DROP TABLE IF EXISTS SDTerms;

CREATE TABLE SDTerms(
    term VARCHAR,
    PRIMARY KEY(term)
) AS
    SELECT DISTINCT source term FROM SD WHERE weight>=0.7
    UNION
    SELECT DISTINCT target term FROM SD WHERE weight>=0.7;

In [9]:
SELECT S.term name, M.l1 mesh FROM SDTerms S, Mesh M WHERE S.term = M.term LIMIT 10;

In [10]:
CALL CSVWRITE('symptoms-disease-g08-nodes.csv',
              'SELECT S.term name, M.l1 mesh FROM SDTerms S, Mesh M WHERE S.term = M.term',
              'fieldDelimiter=')

1918

In [11]:
SELECT * FROM Mesh WHERE tree='C14';